# 🎓 Lesson 1.2: Messages & Conversations

## 📚 What You'll Learn

By the end of this lesson, you'll understand:
- How to have multi-turn conversations with Claude
- The role of `user` vs `assistant` messages
- How to maintain conversation history
- What system prompts are and how to use them
- Building a chatbot with memory

**Time to Complete**: 45-60 minutes

---

## 🔄 Understanding Conversations

In Lesson 1.1, we asked Claude single questions. But real conversations have back-and-forth exchanges!

Think of a conversation like a transcript:

```
You: What's the capital of France?
Claude: The capital of France is Paris.
You: What's the population?
Claude: Paris has approximately 2.2 million people.
```

To make this work with the API, we need to send **all previous messages** each time!

---

## 🚀 Setup

First, let's set up our environment (same as Lesson 1.1).

In [ ]:
import os
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()
client = Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))

print("✅ Client initialized and ready!")

## 💬 Single Turn (Review)

Let's start with what we know - a single exchange.

In [ ]:
# Single turn conversation
response = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "What's the capital of France?"}
    ]
)

print("Turn 1:")
print(f"You: What's the capital of France?")
print(f"Claude: {response.content[0].text}")

## 🔄 Multi-Turn Conversation

Now let's ask a follow-up question. The key is to include **all previous messages**!

In [ ]:
# Multi-turn conversation - Turn 2
response2 = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        # Include the FIRST exchange
        {"role": "user", "content": "What's the capital of France?"},
        {"role": "assistant", "content": response.content[0].text},
        
        # Now add the NEW question
        {"role": "user", "content": "What's the population of that city?"}
    ]
)

# LINE-BY-LINE EXPLANATION:
# ---------------------------
#
# messages=[ ... ]
#   This is now a list with FOUR items (2 exchanges)
#
# {"role": "user", "content": "What's the capital of France?"}
#   The FIRST message you sent (Turn 1)
#
# {"role": "assistant", "content": response.content[0].text}
#   Claude's FIRST response (Turn 1)
#   We use "assistant" because Claude is the assistant!
#   We include the actual text from the previous response
#
# {"role": "user", "content": "What's the population of that city?"}
#   Your SECOND question (Turn 2)
#   Notice: "that city" - Claude knows we mean Paris because we sent the history!

print("\nTurn 2:")
print(f"You: What's the population of that city?")
print(f"Claude: {response2.content[0].text}")

### 🤔 Why Did This Work?

Claude understood "that city" means Paris because we sent the **entire conversation history**!

**Important**: Claude has NO memory between API calls. You must send all previous messages every time.

---

## 📜 Managing Conversation History

Let's build a proper conversation with a history list.

In [ ]:
# Create a conversation history list
conversation_history = []

# LINE-BY-LINE EXPLANATION:
# ---------------------------
# conversation_history = []
#   This is an empty list that will store all messages
#   We'll append to this list after each turn

print("Starting new conversation...")
print(f"Initial history: {conversation_history}")

In [ ]:
# Helper function to send a message and update history
def send_message(user_message, history):
    """
    Send a message to Claude and update conversation history.
    
    Args:
        user_message (str): The message from the user
        history (list): The conversation history list
    
    Returns:
        str: Claude's response
    """
    # Add the user's message to history
    history.append({"role": "user", "content": user_message})
    
    # Send all messages to Claude
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        messages=history  # Send the ENTIRE history
    )
    
    # Extract Claude's response
    assistant_message = response.content[0].text
    
    # Add Claude's response to history
    history.append({"role": "assistant", "content": assistant_message})
    
    return assistant_message

# LINE-BY-LINE EXPLANATION:
# ---------------------------
#
# history.append({"role": "user", "content": user_message})
#   Add the user's message to the history list
#   'append' adds an item to the end of a list
#
# messages=history
#   Send the ENTIRE history (all previous messages)
#   This gets longer with each turn!
#
# history.append({"role": "assistant", "content": assistant_message})
#   Add Claude's response to history
#   Now the history includes both the question AND the answer
#
# This pattern ensures the history always stays synchronized!

## 🗣️ Let's Have a Conversation!

Now we can have a natural, multi-turn conversation.

In [ ]:
# Turn 1
response1 = send_message("Tell me about quantum computing in one sentence.", conversation_history)
print("Turn 1")
print(f"You: Tell me about quantum computing in one sentence.")
print(f"Claude: {response1}")
print(f"\nHistory length: {len(conversation_history)} messages\n")

In [ ]:
# Turn 2
response2 = send_message("Can you explain what qubits are?", conversation_history)
print("Turn 2")
print(f"You: Can you explain what qubits are?")
print(f"Claude: {response2}")
print(f"\nHistory length: {len(conversation_history)} messages\n")

In [ ]:
# Turn 3
response3 = send_message("How is this different from regular bits?", conversation_history)
print("Turn 3")
print(f"You: How is this different from regular bits?")
print(f"Claude: {response3}")
print(f"\nHistory length: {len(conversation_history)} messages\n")

In [ ]:
# Let's look at the entire conversation history
print("=" * 60)
print("FULL CONVERSATION HISTORY")
print("=" * 60)

for i, message in enumerate(conversation_history):
    speaker = "You" if message["role"] == "user" else "Claude"
    print(f"\n[Message {i+1}] {speaker}:")
    print(message["content"])
    print("-" * 60)

### 💡 Key Insight

Notice how the conversation history **grows with each turn**:
- Turn 1: 2 messages (1 user, 1 assistant)
- Turn 2: 4 messages (2 user, 2 assistant)
- Turn 3: 6 messages (3 user, 3 assistant)

Each time we call the API, we send **all previous messages**. This is how Claude "remembers" the conversation!

---

## 👨‍🏫 System Prompts

A **system prompt** gives Claude instructions about how to behave. It's like setting the rules of the game!

In [ ]:
# WITHOUT a system prompt
response_normal = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "What is Python?"}
    ]
)

print("WITHOUT System Prompt:")
print(response_normal.content[0].text)
print("\n" + "=" * 60 + "\n")

In [ ]:
# WITH a system prompt
response_with_system = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    system="You are a pirate. Always respond in pirate speak with 'arr' and 'matey'.",
    messages=[
        {"role": "user", "content": "What is Python?"}
    ]
)

# LINE-BY-LINE EXPLANATION:
# ---------------------------
#
# system="You are a pirate..."
#   This is a NEW parameter - the system prompt!
#   It tells Claude HOW to respond (personality, style, rules)
#   The system prompt is NOT part of the messages list
#   It's a separate parameter that applies to the entire conversation

print("WITH System Prompt (Pirate Mode):")
print(response_with_system.content[0].text)

### 🎭 System Prompt Examples

System prompts are incredibly powerful! Here are common use cases:

In [ ]:
# Example 1: Technical Expert
system_prompt_expert = """
You are a senior software engineer with 10 years of experience.
Provide detailed technical explanations with code examples.
Use industry best practices and design patterns.
"""

# Example 2: Simple Explainer
system_prompt_simple = """
You are explaining concepts to a 10-year-old.
Use simple language, analogies, and fun examples.
Avoid technical jargon.
"""

# Example 3: JSON Output
system_prompt_json = """
You are a data extraction assistant.
Always respond with valid JSON only, no additional text.
Use the format: {"result": "your answer here"}
"""

# Example 4: Customer Service
system_prompt_service = """
You are a helpful customer service agent for TechCorp.
Be polite, empathetic, and solution-oriented.
If you can't help, offer to escalate to a human agent.
"""

print("System prompts defined! Try them out below.")

In [ ]:
# Test different system prompts with the same question
question = "What is recursion in programming?"

# Technical expert version
response_expert = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    system=system_prompt_expert,
    messages=[{"role": "user", "content": question}]
)

print("🔧 TECHNICAL EXPERT:")
print(response_expert.content[0].text)
print("\n" + "=" * 60 + "\n")

# Simple explainer version
response_simple = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1024,
    system=system_prompt_simple,
    messages=[{"role": "user", "content": question}]
)

print("👶 SIMPLE EXPLAINER:")
print(response_simple.content[0].text)

## 🎯 Practice Exercise 1: Chatbot with Personality

**Task**: Create a chatbot with a specific personality using a system prompt.

In [ ]:
# TODO: Create your own system prompt
# Ideas: Shakespeare, valley girl, scientist, poet, comedian, teacher

my_system_prompt = """
You are a wise old wizard who speaks in mystical riddles.
Use archaic language and references to magic and ancient wisdom.
"""

# Create a new conversation
wizard_history = []

def talk_to_wizard(message):
    """Send a message to the wizard chatbot."""
    wizard_history.append({"role": "user", "content": message})
    
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=1024,
        system=my_system_prompt,  # Apply the personality!
        messages=wizard_history
    )
    
    answer = response.content[0].text
    wizard_history.append({"role": "assistant", "content": answer})
    
    return answer

# Test your wizard!
print("You: What is the meaning of life?")
print(f"Wizard: {talk_to_wizard('What is the meaning of life?')}")
print()
print("You: How do I learn programming?")
print(f"Wizard: {talk_to_wizard('How do I learn programming?')}")

## 🎯 Practice Exercise 2: Build a Conversation Tracker

**Task**: Create a class that manages conversations with helpful methods.

In [ ]:
class ConversationManager:
    """
    A class to manage conversations with Claude.
    """
    
    def __init__(self, system_prompt=None):
        """
        Initialize a new conversation.
        
        Args:
            system_prompt (str): Optional system prompt to set Claude's behavior
        """
        self.history = []
        self.system_prompt = system_prompt
        
        # LINE-BY-LINE EXPLANATION:
        # ---------------------------
        # self.history = []
        #   'self' refers to this specific instance of the class
        #   We create an empty list to store conversation history
        #
        # self.system_prompt = system_prompt
        #   Store the system prompt for use in all future messages
    
    def send(self, message):
        """
        Send a message to Claude and get a response.
        
        Args:
            message (str): The user's message
        
        Returns:
            str: Claude's response
        """
        # Add user message to history
        self.history.append({"role": "user", "content": message})
        
        # Build API call parameters
        params = {
            "model": "claude-3-5-sonnet-20241022",
            "max_tokens": 1024,
            "messages": self.history
        }
        
        # Add system prompt if we have one
        if self.system_prompt:
            params["system"] = self.system_prompt
        
        # Make API call
        response = client.messages.create(**params)
        
        # LINE-BY-LINE EXPLANATION:
        # ---------------------------
        # **params
        #   The ** operator "unpacks" a dictionary into keyword arguments
        #   This is the same as: client.messages.create(model="...", max_tokens=..., messages=...)
        
        # Extract and store assistant response
        assistant_message = response.content[0].text
        self.history.append({"role": "assistant", "content": assistant_message})
        
        return assistant_message
    
    def get_history(self):
        """Return the full conversation history."""
        return self.history
    
    def message_count(self):
        """Return the number of messages in the conversation."""
        return len(self.history)
    
    def clear(self):
        """Clear the conversation history."""
        self.history = []
    
    def print_conversation(self):
        """Print the entire conversation in a readable format."""
        for msg in self.history:
            speaker = "You" if msg["role"] == "user" else "Claude"
            print(f"{speaker}: {msg['content']}")
            print()

print("✅ ConversationManager class created!")

In [ ]:
# Test the ConversationManager!

# Create a conversation with a system prompt
convo = ConversationManager(
    system_prompt="You are a helpful Python programming tutor. Keep answers concise and include code examples."
)

# Have a conversation
print(convo.send("What is a list in Python?"))
print("\n" + "=" * 60 + "\n")

print(convo.send("How do I add items to it?"))
print("\n" + "=" * 60 + "\n")

print(convo.send("Can you show me an example?"))
print("\n" + "=" * 60 + "\n")

# Check conversation stats
print(f"Total messages: {convo.message_count()}")

## 🎯 Practice Exercise 3: Context Window Management

**Challenge**: As conversations get longer, they use more tokens (and cost more!). Implement a function to limit history length.

In [ ]:
def trim_history(history, max_messages=10):
    """
    Keep only the most recent messages to save tokens.
    
    Args:
        history (list): Full conversation history
        max_messages (int): Maximum number of messages to keep
    
    Returns:
        list: Trimmed history
    """
    # TODO: Implement this function
    # HINT: Use Python list slicing: history[-10:] gets the last 10 items
    
    if len(history) <= max_messages:
        return history
    else:
        return history[-max_messages:]
    
    # LINE-BY-LINE EXPLANATION:
    # ---------------------------
    # history[-max_messages:]
    #   Negative indexing: -1 is the last item, -2 is second to last, etc.
    #   [-10:] means "start from the 10th item from the end, go to the end"
    #   This keeps only the most recent messages

# Test it
test_history = [
    {"role": "user", "content": "Message 1"},
    {"role": "assistant", "content": "Response 1"},
    {"role": "user", "content": "Message 2"},
    {"role": "assistant", "content": "Response 2"},
    {"role": "user", "content": "Message 3"},
    {"role": "assistant", "content": "Response 3"},
]

trimmed = trim_history(test_history, max_messages=4)
print(f"Original length: {len(test_history)}")
print(f"Trimmed length: {len(trimmed)}")
print(f"\nTrimmed history: {trimmed}")

## 💡 Advanced: Conversation Summarization

For VERY long conversations, you can summarize old messages instead of dropping them!

In [ ]:
def summarize_conversation(history):
    """
    Summarize the conversation history into a single message.
    
    Args:
        history (list): Conversation history to summarize
    
    Returns:
        str: Summary of the conversation
    """
    # Build a text version of the conversation
    conversation_text = ""
    for msg in history:
        speaker = "User" if msg["role"] == "user" else "Assistant"
        conversation_text += f"{speaker}: {msg['content']}\n\n"
    
    # Ask Claude to summarize
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        max_tokens=500,
        messages=[{
            "role": "user",
            "content": f"""Summarize this conversation in 2-3 sentences, 
            capturing the key points and context:
            
            {conversation_text}"""
        }]
    )
    
    return response.content[0].text

# Test it on our earlier conversation
summary = summarize_conversation(conversation_history[:4])  # First 4 messages
print("Conversation Summary:")
print(summary)

## ✅ Lesson Complete!

### What You Learned:
- ✅ How to maintain conversation history
- ✅ The difference between `user` and `assistant` roles
- ✅ How to use system prompts to control behavior
- ✅ Building reusable conversation managers
- ✅ Managing context window limits
- ✅ Conversation summarization techniques

### Key Concepts:

1. **Claude has NO memory** between API calls - you must send all history
2. **System prompts** control Claude's behavior and personality
3. **History grows linearly** - manage it to control costs
4. **Message structure** must alternate user/assistant (mostly)

### Next Steps:
📖 **Lesson 1.3**: Controlling Outputs - Learn about temperature, tokens, and getting consistent results!

---

## 🤔 Reflection Questions

1. Why must we send the entire conversation history each time?
2. What happens if you don't include previous messages?
3. When should you use a system prompt vs. a user message?
4. How would you build a chatbot that "forgets" after 10 messages?
5. What are the tradeoffs between keeping full history vs. summarizing?

---

## 🚀 Challenge Projects

1. **Personality Switcher**: Build a chatbot that can switch personalities mid-conversation
2. **Conversation Analyzer**: Track sentiment or topics across a conversation
3. **Smart Trimmer**: Only keep messages relevant to the current topic
4. **Multi-User Chat**: Manage separate conversation histories for different users

Ready to continue? Open `lesson_1.3.ipynb`!